In [1]:
import numpy as np
from scipy.spatial.distance import pdist, squareform
from scipy.stats import norm
import pandas as pd
import statistics
import math

In [2]:
# ------------ Here you can upload your data ------------ #
path =  # Path of file
title =  # Give your file a name
TS = pd.read_excel(path+title+".xlsx")
ts = TS.ts.values[0:20000]
labels = TS.label.values[0:20000]

# Time Series Length
N = len(ts)

# Here you can enter the subsequence lengths you want to examine
SSQ = [int(0.005*N), int(0.01*N), int(0.02*N), int(0.03*N), int(0.05*N)] 

print("Time series length:", N)
print("Subsequence lengths:", SSQ)

Time series length: 20000
Subsequence lengths: [100, 200, 400, 600, 1000]


In [3]:
def distancedistributions(ts, ssqlength):
    global D1, D2, SF1, SF2, M
    M = int(ssqlength)
    
    res1 = np.zeros((N-M+1, M))
    res2 = np.zeros((N-M+1, M))
    for k in range(0, N-M+1):
        mean1 = np.mean(ts[k:k+M-1])
        sd1 = np.std(ts[k:k+M-1])
        for j in range(0,M):
            res2[k,j] = ts[k+j-1]
            res1[k,j] = (ts[k+j-1]-mean1)/sd1
            
    D1 = pdist(res1, 'euclidean') # EDz
    D2 = pdist(res2, 'euclidean') # ED
     
    SF1 = squareform(D1)
    SF2 = squareform(D2)
    
    # Exclusion zone - based on code from Github Matrix Profile
    for i in range(N-M+1):   
        q = int(M/2)+1
        L = max(i - q, 0)
        LL = min(i + q, N-M+1) 
        SF1[i][L:LL] = 1000
        SF2[i][L:LL] = 1000


In [4]:
def meaningfulMSSs(): 
    global MM1, MM2
    mindist1 = np.nanmin(SF1, axis = 1)
    mindist2 = np.nanmin(SF2, axis = 1)
    index1 = np.argmin(SF1, axis = 1)
    index2 = np.argmin(SF2, axis = 1)
    
    MM1 = list()
    MM2 = list()
    for j in range(len(mindist1)):
        b1 = index1[j]
        b2 = index2[j]
        count1 = 0
        count2 = 0
        for ll in range(M):
            if labels[j+ll] == labels[b1+ll]:
                count1 = count1 + 1
            if labels[j+ll] == labels[b2+ll]:
                count2 = count2 + 1    

        MM1.append(round(count1/M, 3))
        MM2.append(round(count2/M, 3))
        

In [5]:
meaningful1 = list()
meaningful2 = list()

for k in range(len(SSQ)):
    ssqlength = SSQ[k]
    distancedistributions(ts, ssqlength)
    meaningfulMSSs()
    meaningful1.append(round(statistics.mean(MM1)*100,1))
    meaningful2.append(round(statistics.mean(MM2)*100,1))
    print("m = ", ssqlength, ", EDz = ", meaningful1[-1], ", ED = ", meaningful2[-1])

m =  100 , EDz =  85.0 , ED =  14.5
m =  200 , EDz =  84.7 , ED =  14.3
m =  400 , EDz =  84.0 , ED =  13.9
m =  600 , EDz =  83.3 , ED =  13.5
m =  1000 , EDz =  83.0 , ED =  12.8
